In [155]:
#Import packages
import numpy as np
import scipy as sp
import matplotlib as ml
import os
import sys
flopypth = os.path.join('..', '..', 'flopy')
if flopypth not in sys.path:
    sys.path.append(flopypth)
import flopy
import flopy.utils.binaryfile as bf
import subprocess
import matplotlib.pyplot as plt
import shutil
from scipy.stats import uniform
from pyDOE import *

In [210]:
hk_min = 1e-2
hk_max = 10
vka_min = 1e-3
vka_max = 1
numWells = 2
pump_min = -50000.
pump_max = -3000.
sampleSize = 10
time_min = 3000
time_max = 4000
n = 2

In [13]:
# Generate arrays of hk and vka
hk = np.linspace(hk_min, hk_max, num=sampleSize)
vka = np.linspace(vka_min,vka_max,num=sampleSize)
hk
vka

array([ 0.001,  0.112,  0.223,  0.334,  0.445,  0.556,  0.667,  0.778,
        0.889,  1.   ])

In [8]:
list(range(numWells))

[0, 1]

In [18]:
params_in_sample = ['hk', 'vka']
for n in range(numWells):
    params_in_sample.append('pump_rate_' + str(n+1))
params_in_sample

['hk', 'vka', 'pump_rate_1', 'pump_rate_2']

In [69]:
pump = np.zeros([numWells, sampleSize])
for n in range(numWells):
    pump[n,:] = np.linspace(pump_min, pump_max, num=sampleSize)
        
pumpSplit

[array([[  3000.        ,   8222.22222222,  13444.44444444,  18666.66666667,
          23888.88888889,  29111.11111111,  34333.33333333,  39555.55555556,
          44777.77777778,  50000.        ]]),
 array([[  3000.        ,   8222.22222222,  13444.44444444,  18666.66666667,
          23888.88888889,  29111.11111111,  34333.33333333,  39555.55555556,
          44777.77777778,  50000.        ]])]

In [71]:
pumpSplit = np.split(pump,numWells)
param_sample = np.stack([hk, vka])
for i in range(numWells):
    param_sample = np.append(param_sample,pumpSplit[i], axis=0)
param_sample

array([[  1.00000000e-02,   1.12000000e+00,   2.23000000e+00,
          3.34000000e+00,   4.45000000e+00,   5.56000000e+00,
          6.67000000e+00,   7.78000000e+00,   8.89000000e+00,
          1.00000000e+01],
       [  1.00000000e-03,   1.12000000e-01,   2.23000000e-01,
          3.34000000e-01,   4.45000000e-01,   5.56000000e-01,
          6.67000000e-01,   7.78000000e-01,   8.89000000e-01,
          1.00000000e+00],
       [  3.00000000e+03,   8.22222222e+03,   1.34444444e+04,
          1.86666667e+04,   2.38888889e+04,   2.91111111e+04,
          3.43333333e+04,   3.95555556e+04,   4.47777778e+04,
          5.00000000e+04],
       [  3.00000000e+03,   8.22222222e+03,   1.34444444e+04,
          1.86666667e+04,   2.38888889e+04,   2.91111111e+04,
          3.43333333e+04,   3.95555556e+04,   4.47777778e+04,
          5.00000000e+04]])

In [73]:
# Generate list of all paramters
params_in_sample = ['hk', 'vka']
for n in range(numWells):
    params_in_sample.append('pump_rate_' + str(n+1))

# Generate arrays of hk and vka
hk = np.linspace(hk_min, hk_max, num=sampleSize)
vka = np.linspace(vka_min,vka_max,num=sampleSize)

# Generate arrays of pumping rate
pump = np.zeros([numWells, sampleSize])
for n in range(numWells):
    pump[n,:] = np.linspace(pump_min, pump_max, num=sampleSize)
    
# Combine to form paramSample
pumpSplit = np.split(pump,numWells)
param_sample = np.stack([hk, vka])
for i in range(numWells):
    param_sample = np.append(param_sample,pumpSplit[i], axis=0)
    
param_sample

array([[  1.00000000e-02,   1.12000000e+00,   2.23000000e+00,
          3.34000000e+00,   4.45000000e+00,   5.56000000e+00,
          6.67000000e+00,   7.78000000e+00,   8.89000000e+00,
          1.00000000e+01],
       [  1.00000000e-03,   1.12000000e-01,   2.23000000e-01,
          3.34000000e-01,   4.45000000e-01,   5.56000000e-01,
          6.67000000e-01,   7.78000000e-01,   8.89000000e-01,
          1.00000000e+00],
       [  3.00000000e+03,   8.22222222e+03,   1.34444444e+04,
          1.86666667e+04,   2.38888889e+04,   2.91111111e+04,
          3.43333333e+04,   3.95555556e+04,   4.47777778e+04,
          5.00000000e+04],
       [  3.00000000e+03,   8.22222222e+03,   1.34444444e+04,
          1.86666667e+04,   2.38888889e+04,   2.91111111e+04,
          3.43333333e+04,   3.95555556e+04,   4.47777778e+04,
          5.00000000e+04]])

In [140]:
from scipy.stats import uniform
scale = hk_max - hk_min
hk_sample = uniform.rvs(loc=hk_min, scale=scale, size=sampleSize)
hk_sample

array([ 9.86352585,  3.0567227 ,  5.1064603 ,  2.47422919,  9.82114895,
        3.85264791,  1.67685073,  5.15602897,  1.45015831,  7.03355182])

In [213]:
# Write function to generate range of inputs for hk, vka, pumping rate in each well
def genParamSamples(hk_min, hk_max, vka_min, vka_max, numWells, pump_min, pump_max, time_min, time_max, sampleSize):
    
    # Generate list of all paramters
    params_in_sample = ['hk', 'vka', 'time']
    for n in range(numWells):
        params_in_sample.append('pump_rate_' + str(n+1))
        
    # Generate LHS samples
    numParam = np.size(params_in_sample)
    lhd = lhs(numParam, samples=sampleSize)
    
    # Generate arrays of hk and vka
    loc = hk_min
    scale = hk_max - hk_min
    hk = uniform(loc=loc, scale=scale).ppf(lhd[:, 0])
    loc = vka_min
    scale = vka_max - vka_min
    vka = uniform(loc=loc, scale=scale).ppf(lhd[:, 1])
    
    # Generate arrays of time
    loc = time_min
    scale = time_max - time_min
    time = uniform(loc=loc, scale=scale).ppf(lhd[:, 2])
    
    # Generate arrays of pumping rate
    pump = np.zeros([numWells, sampleSize])
    loc = pump_min
    scale = pump_max - pump_min
    for n in range(numWells):
        pump[n, :] = uniform(loc=loc, scale=scale).ppf(lhd[:, 3+n])
        
        
    # Combine to form paramSample
    pumpSplit = np.split(pump, numWells)
    param_sample = np.stack([hk, vka, time])
    for i in range(numWells):
        param_sample = np.append(param_sample, pumpSplit[i], axis=0)

    # Create dictionary with samples for each parameter
    params = dict(zip(params_in_sample, param_sample))

    return params

In [214]:
params = genParamSamples(hk_min, hk_max, vka_min, vka_max, numWells, pump_min, pump_max, time_min, time_max, sampleSize)
params

{'hk': array([ 1.48790884,  6.57782124,  8.00677412,  9.15345579,  3.4920507 ,
         4.09671132,  7.52066836,  2.5959285 ,  0.975651  ,  5.786026  ]),
 'pump_rate_1': array([-40431.61189838, -23116.96694447, -18099.28499085, -26510.83635235,
        -42612.37102199,  -3146.57086647, -45593.04775808,  -8137.45519388,
        -34426.96771837, -15596.84788955]),
 'pump_rate_2': array([-15577.49199887, -43008.02316716,  -4088.97111146, -19847.42271296,
        -48177.23350831, -38229.39834795, -23774.56745761, -30676.33297892,
         -8457.34116508, -31588.24452393]),
 'time': array([ 3420.37738037,  3287.95919722,  3944.84911418,  3034.70540675,
         3638.42816229,  3819.32226021,  3391.01223203,  3161.06622547,
         3711.29239899,  3586.54302568]),
 'vka': array([ 0.93616877,  0.31941607,  0.71152071,  0.14357281,  0.03917896,
         0.63049969,  0.87050117,  0.29436692,  0.58177696,  0.45427504])}

In [216]:
perlen = params['time'][i]
perlen

3420.3773803657764

In [200]:
pump = np.zeros([numWells, sampleSize])
loc = pump_min
scale = pump_max - pump_min
for n = (range(numWells-1)
pump[0, :] = uniform(loc=loc, scale=scale).ppf(lhd[:, 2])
pump[1, :] = uniform(loc=loc, scale=scale).ppf(lhd[:, 3])
pump

array([[ -9348.57399292, -33353.90506993, -18154.44794339, -37850.31760968,
         -3677.11370142, -16278.62441971, -27706.17543349, -49204.67923356,
        -23113.47708845, -40814.05886537],
       [ -4443.00280899, -29198.73226084, -26418.06512817, -18574.01939047,
        -49215.50134805, -15914.37984226, -42747.72223013,  -8229.34010933,
        -40503.86392735, -34323.42394513]])